In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# def read_data(adjusted):
#     if adjusted:
#         data_file = pd.read_csv('train_adjusted.csv', encoding='cp950')
#         return data_file
#     else:
#         data_file = pd.read_csv('MOVIES_WITHOUT_ADJUSTED.csv', encoding='cp950')
#         return data_file
def preprocessing(data_file):
    country = []
    genre = []
    date = []
    data_length = len(data_file)
    for i in range(data_length):
        t_country = str(data_file['COUNTRY'][i]).split(',')
        t_genre = str(data_file['IMDB_GENRE'][i]).split(',')
        t_date = str(data_file['DATE_TW'][i]).split('/')
        t_date = [ int(d) for d in t_date]
        country.append(t_country)
        genre.append(t_genre)
        date.append(t_date)
    mlb = MultiLabelBinarizer()
    country = mlb.fit_transform(country)
    genre = mlb.fit_transform(genre)
    date = np.array(date)
    runtime = np.array(data_file['IMDB_RUNTIME']).reshape(data_length, 1)
    dir_detail = np.array(data_file[['DIRECTOR_WINS', 'DIRECTOR_NOMINATIONS', 
                                     'DIRECTOR_RATINGS']])
    star_detail = np.array(data_file[['STAR_1_WINS', 'STAR_1_NOMINATIONS', 
                                      'STAR_1_RATINGS', 'STAR_2_WINS', 
                                      'STAR_2_NOMINATIONS', 'STAR_2_RATINGS', 
                                      'STAR_3_WINS', 'STAR_3_NOMINATIONS', 
                                      'STAR_3_RATINGS']])
    yahoo = np.array(data_file[['YAHOO_EVALUATION', 'YAHOO_VOTER']])
    PTT = np.array(data_file[['PTT_ARTICLE', 'PTT_PUSH', 'PTT_ARROW', 
                              'PTT_PULL', 'PTT_REPLY']])
    youtube = np.array(data_file[['YOUTUBE_VIEW', 'YOUTUBE_LIKE', 'YOUTUBE_DISLIKE']])
    x_train = np.concatenate((country, genre, runtime, dir_detail, star_detail, 
                          yahoo, PTT, youtube), axis=-1)
    rating = np.array(data_file['IMDB_RATING']).reshape(data_length, 1)
    return x_train, rating
def set_class(rating, g_1, g_2):
    y_train = []
    for r in rating:
        temp = 0
        if r >= g_1:
            temp = 0
        elif r >= g_2:
            temp = 1
        else:
            temp = 2
        y_train.append(temp)
    y_train = np.array(y_train)
    return y_train

In [3]:
# read training data
x_train, rating = np.load('x_train.npy'), np.load('y_train.npy')
n_features = x_train.shape[1]

In [4]:
g_1, g_2 = 7.0, 6.0
y_train = set_class(rating, g_1, g_2)

In [5]:
# training normalization
mean = np.mean(x_train, axis=0)
sigma = np.std(x_train, axis=0)
x_train = (x_train-mean)/(sigma + 1e-20)

In [6]:
# read testing data
x_test, test_rating = np.load('x_test.npy'), np.load('y_test.npy')
y_test = set_class(test_rating, g_1, g_2)

# testing normalization
x_test = (x_test-mean)/(sigma + 1e-20)

In [7]:
def create_model(dnn):
    model = Sequential()
    model.add(Dense(32, input_shape=(x_train.shape[1],), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    for units in dnn:
        model.add(Dense(units, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
    model.add(Dense(3, activation='softmax'))
    return model

In [8]:
kfold = KFold(n_splits=10, shuffle=True, random_state=5)
k = 0
test_score = []
use_dnn = True
for train_index, valid_index in kfold.split(x_train):
    k += 1
    X, X_V = x_train[train_index], x_train[valid_index]
    Y, Y_V = y_train[train_index], y_train[valid_index]
    
    if use_dnn:
        # DNN
        Y, Y_V = np_utils.to_categorical(Y), np_utils.to_categorical(Y_V)
        dnn = [32, 32, 32]
        model = create_model(dnn)
        callbacks = []
        callbacks.append(EarlyStopping(monitor='val_loss', patience=3))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X, Y, batch_size=32, validation_data=(X_V, Y_V), 
                  epochs=100, callbacks=callbacks)
        test_score.append(model.predict(x_test))
    else:
        # RF & GradientBoosting
#         clf = GradientBoostingClassifier(n_estimators=40, min_samples_split=40, min_samples_leaf=3,
#                                  max_leaf_nodes=15, max_depth=5, random_state=15)
#         clf.fit(X, Y)
#         test_score.append(clf.predict_proba(x_test))
#         print('kfold ' + str(k) +  ' acc: '+ str(clf.score(X_V, Y_V)))
        # Logistic Regression
        lr = LogisticRegression(penalty='l2',C = 0.001,random_state = 0)
        lr.fit(X, Y)
        test_score.append(lr.predict_proba(x_test))
        print('kfold ' + str(k) +  ' acc: '+ str(lr.score(X_V, Y_V)))

Train on 159 samples, validate on 18 samples
Epoch 1/100
159/159 [==============================] - 2s - loss: 2.1127 - acc: 0.3270 - val_loss: 1.1422 - val_acc: 0.3333
Epoch 2/100
159/159 [==============================] - 0s - loss: 1.9419 - acc: 0.3522 - val_loss: 1.1151 - val_acc: 0.3889
Epoch 3/100
159/159 [==============================] - 0s - loss: 1.6561 - acc: 0.3899 - val_loss: 1.0978 - val_acc: 0.3889
Epoch 4/100
159/159 [==============================] - 0s - loss: 1.5383 - acc: 0.3648 - val_loss: 1.0871 - val_acc: 0.3889
Epoch 5/100
159/159 [==============================] - 0s - loss: 1.5761 - acc: 0.3145 - val_loss: 1.0776 - val_acc: 0.3333
Epoch 6/100
159/159 [==============================] - 0s - loss: 1.7599 - acc: 0.3082 - val_loss: 1.0698 - val_acc: 0.2778
Epoch 7/100
159/159 [==============================] - 0s - loss: 1.5036 - acc: 0.3522 - val_loss: 1.0605 - val_acc: 0.2222
Epoch 8/100
159/159 [==============================] - 0s - loss: 1.4752 - acc: 0.3522 

159/159 [==============================] - 0s - loss: 1.6288 - acc: 0.3082 - val_loss: 1.0437 - val_acc: 0.2778
Epoch 4/100
159/159 [==============================] - 0s - loss: 1.6599 - acc: 0.3145 - val_loss: 1.0460 - val_acc: 0.2222
Epoch 5/100
159/159 [==============================] - 0s - loss: 1.5047 - acc: 0.3585 - val_loss: 1.0455 - val_acc: 0.2778
Train on 159 samples, validate on 18 samples
Epoch 1/100
159/159 [==============================] - 1s - loss: 1.6639 - acc: 0.3585 - val_loss: 1.0375 - val_acc: 0.4444
Epoch 2/100
159/159 [==============================] - 0s - loss: 1.8452 - acc: 0.2956 - val_loss: 1.0430 - val_acc: 0.5000
Epoch 3/100
159/159 [==============================] - 0s - loss: 1.6138 - acc: 0.3648 - val_loss: 1.0450 - val_acc: 0.5000
Epoch 4/100
159/159 [==============================] - 0s - loss: 1.6421 - acc: 0.3396 - val_loss: 1.0466 - val_acc: 0.4444
Epoch 5/100
159/159 [==============================] - 0s - loss: 1.6803 - acc: 0.3019 - val_loss: 

159/159 [==============================] - 0s - loss: 1.1094 - acc: 0.4717 - val_loss: 1.0353 - val_acc: 0.5556
Epoch 16/100
159/159 [==============================] - 0s - loss: 1.2289 - acc: 0.4717 - val_loss: 1.0339 - val_acc: 0.5556
Epoch 17/100
159/159 [==============================] - 0s - loss: 1.2142 - acc: 0.4277 - val_loss: 1.0322 - val_acc: 0.5000
Epoch 18/100
159/159 [==============================] - 0s - loss: 1.0187 - acc: 0.5094 - val_loss: 1.0275 - val_acc: 0.5000
Epoch 19/100
159/159 [==============================] - 0s - loss: 1.1193 - acc: 0.4528 - val_loss: 1.0221 - val_acc: 0.5000
Epoch 20/100
159/159 [==============================] - 0s - loss: 1.1428 - acc: 0.4465 - val_loss: 1.0186 - val_acc: 0.5000
Epoch 21/100
159/159 [==============================] - 0s - loss: 1.1472 - acc: 0.4277 - val_loss: 1.0155 - val_acc: 0.5000
Epoch 22/100
159/159 [==============================] - 0s - loss: 1.1582 - acc: 0.4654 - val_loss: 1.0131 - val_acc: 0.5000
Epoch 23/100


Epoch 44/100
160/160 [==============================] - 0s - loss: 0.9604 - acc: 0.5812 - val_loss: 0.8287 - val_acc: 0.7059
Epoch 45/100
160/160 [==============================] - 0s - loss: 0.8636 - acc: 0.6125 - val_loss: 0.8231 - val_acc: 0.7059
Epoch 46/100
160/160 [==============================] - 0s - loss: 0.8771 - acc: 0.6063 - val_loss: 0.8163 - val_acc: 0.7059
Epoch 47/100
160/160 [==============================] - 0s - loss: 0.8297 - acc: 0.6250 - val_loss: 0.8112 - val_acc: 0.7059
Epoch 48/100
160/160 [==============================] - 0s - loss: 0.9138 - acc: 0.5563 - val_loss: 0.8093 - val_acc: 0.6471
Epoch 49/100
160/160 [==============================] - 0s - loss: 0.8750 - acc: 0.5687 - val_loss: 0.8057 - val_acc: 0.7059
Epoch 50/100
160/160 [==============================] - 0s - loss: 0.9168 - acc: 0.5375 - val_loss: 0.8058 - val_acc: 0.6471
Epoch 51/100
160/160 [==============================] - 0s - loss: 0.7974 - acc: 0.6188 - val_loss: 0.8019 - val_acc: 0.6471


160/160 [==============================] - 0s - loss: 1.4363 - acc: 0.3375 - val_loss: 1.0915 - val_acc: 0.3529
Epoch 10/100
160/160 [==============================] - 0s - loss: 1.3675 - acc: 0.4312 - val_loss: 1.0875 - val_acc: 0.3529
Epoch 11/100
160/160 [==============================] - 0s - loss: 1.3348 - acc: 0.3687 - val_loss: 1.0803 - val_acc: 0.3529
Epoch 12/100
160/160 [==============================] - 0s - loss: 1.2848 - acc: 0.3813 - val_loss: 1.0737 - val_acc: 0.3529
Epoch 13/100
160/160 [==============================] - 0s - loss: 1.1976 - acc: 0.3687 - val_loss: 1.0668 - val_acc: 0.3529
Epoch 14/100
160/160 [==============================] - 0s - loss: 1.2987 - acc: 0.3563 - val_loss: 1.0602 - val_acc: 0.3529
Epoch 15/100
160/160 [==============================] - 0s - loss: 1.3318 - acc: 0.4000 - val_loss: 1.0546 - val_acc: 0.3529
Epoch 16/100
160/160 [==============================] - 0s - loss: 1.2692 - acc: 0.4000 - val_loss: 1.0502 - val_acc: 0.3529
Epoch 17/100


In [9]:
# test accuracy
final_scores = np.zeros((len(x_test), 3))
for i in range(k):
    final_scores += test_score[i]
test_pred = [np.argmax(x) for x in final_scores]
test_acc = [1 if y_test[i] == test_pred[i] else 0 for i in range(len(y_test))]
test_acc = np.sum(test_acc) / len(y_test)
print('test_acc: ' + str(test_acc))

test_acc: 0.55
